In [1]:
import os
import sys
import scipy
from numba import njit, prange
import numpy as np
import scipy.stats as stats
import bayesflow as bf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

import ctypes
from numba.extending import get_cython_function_address
from simulators import levy
from sklearn.covariance import EmpiricalCovariance


# Get a pointer to the C function levy.c
addr_levy= get_cython_function_address("levy", "levy_trial")
functype = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double, ctypes.c_double, 
                            ctypes.c_double, ctypes.c_double, ctypes.c_double,
                            ctypes.c_double, ctypes.c_double, ctypes.c_double, 
                            ctypes.c_double, ctypes.c_double, ctypes.c_int)
levy_trial = functype(addr_levy)

# Suppress scientific notation for floats
np.set_printoptions(suppress=True)
RNG = np.random.default_rng(2023)

2024-06-12 09:26:37.631158: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 09:26:37.651610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 09:26:37.973809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/bayesflow/trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook imp

In [2]:
#Settings
# Path to data
PATH = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/data/prepared_data/'

# Where to save files
PATH_TO_SAVE = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/data/pickle_levy/'

# Load neural networks from checkpoint

In [3]:
PARAM_NAMES = [
    r"$v_{1}$",
    r"$v_{2}$",
    r"$a_{1}$",
    r"$a_{2}$",
    r"$\tau_{correct}$",
    r"$\tau_{error}$",
    r"$\alpha$"
]

prior = bf.simulation.Prior(prior_fun=ps.levy_prior_fun, param_names=PARAM_NAMES)

prior_means, prior_stds = prior.estimate_means_and_stds(n_draws=100000)
prior_means = np.round(prior_means, decimals=1)
prior_stds = np.round(prior_stds, decimals=1)

simulator = bf.simulation.Simulator(simulator_fun=ps.levy_simulator_fun, context_generator=context_gen)

model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator, name="Levy")

summary_net = bf.networks.SetTransformer(input_dim=4, summary_dim=30, name="ddm_summary")

inference_net = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_settings={"dense_args": dict(kernel_regularizer=None), "dropout": False},
    name="ddm_inference")

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net, name="levy_amortizer",
                                            summary_loss_fun='MMD')

trainer = bf.trainers.Trainer(
    generative_model=model, amortizer=amortizer, configurator= ps.configurator, checkpoint_path="levy_model_net2")

INFO:root:Performing 2 pilot runs with the Levy model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 7)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 120, 1)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:Could not determine shape of simulation batchable context. Type appears to be non-array: <class 'list'>,                                    so make sure your input configurator takes cares of that!
2024-06-12 09:26:44.005560: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-12 09:26:44.019235: I tensorflow/com

In [8]:
#This is where the magic happens

# 1. Store all data-set chunk names in a list
datasets = sorted(os.listdir(PATH))

# 2. For each chunk
for dataset_name in datasets:
    
        # 2.1 Load chunk
        loaded_pickle = pickle.load(open(PATH +str(dataset_name), "rb" ))
        
        X_test = loaded_pickle['data_array']
        y_test = loaded_pickle['outcome_array']
        rt_summaries = loaded_pickle['rt_summaries']
        
        print(str(dataset_name)+ " loaded")


        # 2.2 Estimate chunk

        samples_dm = np.concatenate([amortizer.sample(input_dict = {"summary_conditions": x}, n_samples=3000,
                                                  to_numpy=True) for x in np.array_split(X_test, 50)], axis=0)

        samples_dm = samples_dm * prior_stds + prior_means
        
        # Discard negative samples for positively bounded parameters
        samples_dm[:,:,2:7][samples_dm[:,:,2:7]<0] = np.nan 
        
        # Discard samples > 2 for alpha
        samples_dm[:,:,6][samples_dm[:,:,6]>2] = np.nan
        
        print(str(np.round(np.sum(np.isnan(samples_dm))/
                           (np.sum(np.isnan(samples_dm))+np.sum(~np.isnan(samples_dm))),
                           5)*100) +"% out of prior samples rejected")
        print(str(dataset_name)+ " inference done")

        # 2.3 Compute summaries of parameter posteriors: means, medians, stds, Q0.025, Q0.0975
        
        estimates = ps.compute_summaries(samples_dm)

        # Exclude people with less than 1000 proper prior samples for at least one parameter
        estimates[np.sum((np.sum(np.isnan(samples_dm), axis=1)>4000), axis=1)>0,:] = np.nan 
                
        print(str(np.sum((np.sum(np.isnan(samples_dm), axis=1))>4000)) +
              " people with <1000 proper samples (for at least one parameter) excluded.")

        # 2.4 Get empirical Mahalanobis distances for summary statistics provided by network
            
        summary_statistics_empirical = np.concatenate([trainer.amortizer.summary_net (x)
                                                       for x in np.array_split(X_test, 50)], axis=0)
        
        cov = EmpiricalCovariance().fit(summary_statistics_empirical)

        mahalanobis_empirical = cov.mahalanobis(summary_statistics_empirical)
        
        print(str(dataset_name)+ " Mahalanobis check done")

        
        # 2.5 Store everything together (serialized, pickle.dump) as a dict with keys 
        dict_to_store = {'data_array': X_test, 'est_array': estimates, "outcome_array": y_test,
                        'mahalanobis': mahalanobis_empirical, 'rt_summaries': rt_summaries}
        pickle.dump(dict_to_store,
                    open(PATH_TO_SAVE +"estimates_" +str(dataset_name),"wb"))
        print(str(dataset_name)+ " saving done")

# 3. Celebrate

Initial shape: 5604
After removing no age IDs: 4550
After removing invalid IDs: 4550
After excluding zero latency IDs: 4546
After excluding less than 120 trials IDs: 3367
After excluding more than 50% error IDs: 3363
After excluding zero error IDs: 3276
Successfully read datafile 2003iat.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (3276, 120, 4)
Final shape of outcomes chunk:  (3276, 2)
IDs with less than 12 trials >300 ms: (3249, 120, 2)
In prior-cases: (3249,)
2003iat.txt done
Initial shape: 25354
After removing no age IDs: 4225
After removing invalid IDs: 4225
After excluding zero latency IDs: 4220
After excluding less than 120 trials IDs: 2050
After excluding more than 50% error IDs: 2047
After excluding zero error IDs: 1994
Successfully read datafile 2004iat.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (1994, 120, 4)
Final s

Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2007iat4.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2007iat5.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2007iat6.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
Aft

Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2011iat3.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2011iat4.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2011iat5.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
Aft

Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2015MarDeciat5.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2015MarDeciat6.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing invalid IDs: 0
After excluding zero latency IDs: 0
After excluding less than 120 trials IDs: 0
After excluding more than 50% error IDs: 0
After excluding zero error IDs: 0
Successfully read datafile 2015MarDeciat7.txt .
Converting to X and y arrays...
Initial shape: 0
After removing no age IDs: 0
After removing 

After excluding zero latency IDs: 15840
After excluding less than 120 trials IDs: 15800
After excluding more than 50% error IDs: 15789
After excluding zero error IDs: 14776
Successfully read datafile 2017iat10.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (14776, 120, 4)
Final shape of outcomes chunk:  (14776, 2)
IDs with less than 12 trials >300 ms: (14659, 120, 2)
In prior-cases: (14659,)
2017iat10.txt done
Initial shape: 26049
After removing no age IDs: 16072
After removing invalid IDs: 16072
After excluding zero latency IDs: 15941
After excluding less than 120 trials IDs: 15887
After excluding more than 50% error IDs: 15875
After excluding zero error IDs: 14833
Successfully read datafile 2017iat11.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (14833, 120, 4)
Final shape of outcomes chunk:  (14833, 2)
IDs with less than 12 trials

...

After excluding zero latency IDs: 17540
After excluding less than 120 trials IDs: 17514
After excluding more than 50% error IDs: 17509
After excluding zero error IDs: 15504
Successfully read datafile 2020iat12.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (15504, 120, 4)
Final shape of outcomes chunk:  (15504, 2)
IDs with less than 12 trials >300 ms: (15481, 120, 2)
In prior-cases: (15481,)
2020iat12.txt done
Initial shape: 25719
After removing no age IDs: 17748
After removing invalid IDs: 17748
After excluding zero latency IDs: 17684
After excluding less than 120 trials IDs: 17658
After excluding more than 50% error IDs: 17653
After excluding zero error IDs: 15642
Successfully read datafile 2020iat13.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (15642, 120, 4)
Final shape of outcomes chunk:  (15642, 2)
IDs with less than 12 trials

Successfully read datafile 2020iat25.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (7513, 120, 4)
Final shape of outcomes chunk:  (7513, 2)
IDs with less than 12 trials >300 ms: (7476, 120, 2)
In prior-cases: (7476,)
2020iat25.txt done
Initial shape: 25578
After removing no age IDs: 15673
After removing invalid IDs: 15673
After excluding zero latency IDs: 15522
After excluding less than 120 trials IDs: 15491
After excluding more than 50% error IDs: 15473
After excluding zero error IDs: 14511
Successfully read datafile 2020iat3.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (14511, 120, 4)
Final shape of outcomes chunk:  (14511, 2)
IDs with less than 12 trials >300 ms: (14299, 120, 2)
In prior-cases: (14299,)
2020iat3.txt done
Initial shape: 25571
After removing no age IDs: 16529
After removing invalid IDs: 16529
After excluding zero 

In prior-cases: (10458,)
2021iat15.txt done
Initial shape: 25648
After removing no age IDs: 11647
After removing invalid IDs: 11647
After excluding zero latency IDs: 11519
After excluding less than 120 trials IDs: 11494
After excluding more than 50% error IDs: 11487
After excluding zero error IDs: 10556
Successfully read datafile 2021iat16.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (10556, 120, 4)
Final shape of outcomes chunk:  (10556, 2)
IDs with less than 12 trials >300 ms: (10472, 120, 2)
In prior-cases: (10472,)
2021iat16.txt done
Initial shape: 25754
After removing no age IDs: 11603
After removing invalid IDs: 11603
After excluding zero latency IDs: 11466
After excluding less than 120 trials IDs: 11462
After excluding more than 50% error IDs: 11452
After excluding zero error IDs: 10514
Successfully read datafile 2021iat17.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more

After excluding zero latency IDs: 11610
After excluding less than 120 trials IDs: 11593
After excluding more than 50% error IDs: 11579
After excluding zero error IDs: 10632
Successfully read datafile 20221iat2.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (10632, 120, 4)
Final shape of outcomes chunk:  (10632, 2)
IDs with less than 12 trials >300 ms: (10559, 120, 2)
In prior-cases: (10559,)
20221iat2.txt done
Initial shape: 25567
After removing no age IDs: 21546
After removing invalid IDs: 21546
After excluding zero latency IDs: 21282
After excluding less than 120 trials IDs: 20981
After excluding more than 50% error IDs: 20956
After excluding zero error IDs: 19309
Successfully read datafile 20221iat3.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (19309, 120, 4)
Final shape of outcomes chunk:  (19309, 2)
IDs with less than 12 trials

After excluding zero error IDs: 22750
Successfully read datafile 20231iat2.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (22750, 120, 4)
Final shape of outcomes chunk:  (22750, 2)
IDs with less than 12 trials >300 ms: (22300, 120, 2)
In prior-cases: (22300,)
20231iat2.txt done
Initial shape: 25795
After removing no age IDs: 25785
After removing invalid IDs: 25785
After excluding zero latency IDs: 25768
After excluding less than 120 trials IDs: 24855
After excluding more than 50% error IDs: 24763
After excluding zero error IDs: 23034
Successfully read datafile 20231iat3.txt .
Converting to X and y arrays...
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (23034, 120, 4)
Final shape of outcomes chunk:  (23034, 2)
IDs with less than 12 trials >300 ms: (22540, 120, 2)
In prior-cases: (22540,)
20231iat3.txt done
Initial shape: 25795
After removing no age IDs: 25789
After remov

In [ ]:
df = pd.DataFrame()
datasets = os.listdir(PATH_TO_SAVE)

for dataset in datasets:
        pickles = pickle.load(open(PATH_TO_SAVE +str(dataset), "rb" ))
        df_oneset = np.concatenate((pickles['est_array'],pickles['outcome_array'],pickles['rt_summaries'],
                                   np.expand_dims(pickles['mahalanobis'], axis=1)), axis=1)
        df = pd.concat([df, pd.DataFrame(data = df_oneset)])
        df["dataset"] = str(dataset)      
        
        print(str(dataset)+" done")

df.columns = ["v_congruent", "v_incongruent", "a_congruent", "a_incongruent",
                 "tplus", "tminus", "alpha",
                "v_congruent_median", "v_incongruent_median", "a_congruent_median", "a_incongruent_median",
                 "tplus_median","tminus_median", "alpha_median",
                "v_congruent_std", "v_incongruent_std", "a_congruent_std", "a_incongruent_std",
                 "tplus_std","tminus_std", "alpha_std", 
                "v_congruent_q025", "v_incongruent_q025", "a_congruent_q025", "a_incongruent_q025",
                 "tplus_q025","tminus_q025", "alpha_q025", 
                "v_congruent_q975", "v_incongruent_q975", "a_congruent_q975", "a_incongruent_q975",
                 "tplus_q975","tminus_q975", "alpha_q975", 
              
                  "session_id", "age",
              
                 "congruent_rt_correct", "congruent_rt_error", "congruent_accuracy",
                "incongruent_rt_correct", "incongruent_rt_error","incongruent_accuracy",
                "word_rt_correct","word_rt_error","word_accuracy",
                "picture_rt_correct","picture_rt_error","picture_accuracy",
              
                 "mahalanobis_distance", "dataset"]

df.to_csv("df_levy.csv", index=False)